In [1]:
import pandas as pd
import geopandas as gpd
%matplotlib inline
import numpy as np
from shapely.geometry import Point
import requests
import json
import getpass

In [2]:
villa_lobos = gpd.read_file("../../../data/Cidade de São Paulo/Parques/villa_lobos.geojson")
buffer_villa_lobos = gpd.read_file("../../../data/Cidade de São Paulo/Parques/villa_lobos_buffer.geojson")

In [45]:
def get_Cell_IDs(polygon_gdf):
    
    # Get the bounding box of the polygon
    minx, miny, maxx, maxy = polygon_gdf.geometry.total_bounds
    
    # Generate a grid of points covering the bounding box
    x, y = np.meshgrid(np.arange(minx, maxx, 0.001), np.arange(miny, maxy, 0.001))
    grid_points = gpd.GeoDataFrame({'geometry': [Point(x, y) for x, y in zip(x.flatten(), y.flatten())]})
    
    # Determine which points are inside the polygon
    inside = grid_points.within(polygon_gdf.iloc[0].geometry)
    
    # Extract the latitude and longitude of the points that are inside the polygon
    lon_lat = grid_points[inside].apply(lambda x: x.geometry.coords[0], axis=1).values
    
    # Convert the lat_lon list of tuples into a GeoDataFrame
    points = gpd.GeoDataFrame(geometry=[Point(lat, lon) for lat, lon in lon_lat])

    # Set the coordinate reference system (CRS) of the GeoDataFrame to WGS 84 (EPSG 4326)
    points.crs = {'init': 'epsg:4326'}
    
    # API script:
    
    root = "https://api.claro-br.kidodynamics.com/v1/"

    # Login
    user = input('Enter username:')
    password = getpass.getpass('Pass')
    
    login_response = requests.post(
        root + 'users/login',
        headers={
            'accept': 'application/json',
            'Content-Type': 'application/x-www-form-urlencoded',
        },
        data=f'grant_type=&username={user}&password={password}'
    )
    token = login_response.json()['access_token']
    
    def headers(token):
        return {
            'accept': 'application/json',
            'Authorization': f"Bearer {token}"
        }
    
    # List to store the cell IDs
    cell_ids = []

    # Set to store the collected cell IDs
    collected_cell_ids = set()
    
    # Loop through each latitude-longitude pair in lat_lon
    for lon, lat in lon_lat:
        # Format the API URL with the latitude and longitude
        url = f"https://api.claro-br.kidodynamics.com/v1/grid/cell_from_point?lat={lat}&lon={lon}"
    
        # Check if the request was successful
        #if response.status_code == 200:
            # Extract the cell ID from the response JSON
        response = requests.get(url)
    
        data = json.loads(response.text)
        cell_id = data["data"][0]["value"]
        
        # Check if the cell ID has already been collected
        if cell_id not in collected_cell_ids:
            # Append the cell ID to the cell_ids list
            cell_ids.append(cell_id)
            
            # Add the cell ID to the collected_cell_ids set
            collected_cell_ids.add(cell_id)
            
    return(collected_cell_ids)

In [46]:
cells = get_Cell_IDs(buffer_villa_lobos)

C:\Users\gabri\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


Enter username:gabrieljunqueirasouza@hotmail.com
Pass········


In [21]:
#first try
cells

{18505, 47652, 56833, 68676, 86100, 148628, 200132, 240236}

In [43]:
#second try
cells

{3955,
 18505,
 39669,
 47652,
 56833,
 57045,
 68676,
 84777,
 86100,
 87207,
 99503,
 127553,
 148628,
 158030,
 200132,
 201703,
 211037,
 217324,
 217541,
 226891,
 240236}

In [47]:
#third try
cells

{18505,
 47652,
 56833,
 68676,
 84777,
 86100,
 127553,
 148628,
 200132,
 211037,
 226891,
 240236}